# RL for Maximizing Q_fusion

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import gymtorax
import gymnasium as gym
from gymnasium import spaces
import gymtorax.action_handler as ah
import gymtorax.observation_handler as oh
from gymtorax.envs.base_env import BaseEnv
from gymtorax import rewards as torax_reward
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.logger import configure
from stable_baselines3.common.vec_env import DummyVecEnv
import time
import logging
import warnings
import os
from datetime import datetime

In [2]:
warnings.filterwarnings("ignore")
logging.getLogger("gymtorax").setLevel(logging.ERROR)
logging.getLogger('gymnasium').setLevel(logging.ERROR)
logging.getLogger('gym').setLevel(logging.ERROR)
logging.getLogger("gymtorax.action_handler").setLevel(logging.ERROR)

## Import State Space

Our RL agent will only have partial observability into the total state space provided by the TORAX simulator. Here, we import the states we want our agent to be able to observe from a CSV file listing all of the state parameters in the state space. We will then integrate the states that we wish for our agent to observe into our training environment.

In [3]:
# Extract states that are meant to be observable to the RL agent
state_space_csv = "Gym_TORAX_IterHybrid-v0_env - State Space.csv"
df = pd.read_csv(state_space_csv)
observable_df = df[df["OBSERVABLE?"] == "Yes"][["NAME", "TYPE"]].dropna()

# Profile (vector) variables
OBS_PROFILES = (
    observable_df[observable_df["TYPE"] == "vector"]["NAME"]
    .dropna()
    .tolist()
)

# Scalar variables
OBS_SCALARS = (
    observable_df[observable_df["TYPE"] != "vector"]["NAME"]
    .dropna()
    .tolist()
)

print("OBS_PROFILES =", OBS_PROFILES)
print("OBS_SCALARS =", OBS_SCALARS)

OBS_PROFILES = ['T_e', 'T_i', 'n_e', 'psi', 'psi_norm', 'spr', 'R_in', 'R_out', 'delta', 'delta_lower', 'delta_upper', 'elongation', 'magnetic_shear', 'q', 'p_ohmic_e']
OBS_SCALARS = ['H20', 'H89P', 'H97L', 'H98', 'Q_fusion', 'beta_N', 'beta_pol', 'beta_tor', 'dW_thermal_dt', 'tau_E', 'E_aux', 'E_fusion', 'P_alpha_total', 'P_aux_total', 'P_cyclotron_e', 'P_SOL_total', 'B_0', 'R_major', 'a_minor', 'q95', 'q_min', 'rho_q_min', 'S_gas_puff', 'S_generic_particle', 'S_pellet', 'S_total', 'W_pol', 'W_thermal_e', 'W_thermal_i', 'W_thermal_total', 'fgw_n_e_line_avg', 'fgw_n_e_volume_avg', 'n_e_line_avg', 'n_e_volume_avg', 'A_i', 'A_impurity', 'I_aux_generic', 'I_bootstrap', 'I_ecrh', 'P_icrh_total', 'Phi_b', 'Phi_b_dot', 'li3', 'v_loop_lcfs']


## Define Training Environment

Here we implement the Gymnasium training environment. The environment includes the state space, the observation space (a subset of the state space consisting of the state parameters that the RL agent can observe), the action space, and the reward function. We have modified Gym-TORAX's existing base ITER hybrid environment. Our changes to the existing environment are:

1) We have hidden the majority of the state parameters from the RL agent. Only the parameters imported from the CSV file are visible to the agent at any time step.
2) We have modified the reward function to suit our custom objective.

In [4]:
"""
Config for ITER hybrid scenario based parameters with nonlinear solver.

ITER hybrid scenario based (roughly) on van Mulders Nucl. Fusion 2021.
With Newton-Raphson solver and adaptive timestep (backtracking)
"""

_NBI_W_TO_MA = 1 / 16e6  # rough estimate of NBI heating power to current drive
W_to_Ne_ratio = 0

# No NBI during rampup. Rampup all NBI power between 99-100 seconds
nbi_times = np.array([0, 99, 100])
nbi_powers = np.array([0, 0, 33e6])
nbi_cd = nbi_powers * _NBI_W_TO_MA

# Gaussian prescription of "NBI" deposition profiles and fractional deposition
r_nbi = 0.25
w_nbi = 0.25
el_heat_fraction = 0.66

# No ECCD power for this config (but kept here for future flexibility)
eccd_power = {0: 0, 99: 0, 100: 20.0e6}


CONFIG = {
    "plasma_composition": {
        "main_ion": {"D": 0.5, "T": 0.5},  # (bundled isotope average)
        "impurity": {"Ne": 1 - W_to_Ne_ratio, "W": W_to_Ne_ratio},
        "Z_eff": {0.0: {0.0: 2.0, 1.0: 2.0}},  # sets impurity densities
    },
    "profile_conditions": {
        "Ip": {0: 3e6, 100: 12.5e6},  # total plasma current in MA
        "T_i": {0.0: {0.0: 6.0, 1.0: 0.2}},  # T_i initial condition
        "T_i_right_bc": 0.2,  # T_i boundary condition
        "T_e": {0.0: {0.0: 6.0, 1.0: 0.2}},  # T_e initial condition
        "T_e_right_bc": 0.2,  # T_e boundary condition
        "n_e_right_bc_is_fGW": True,
        "n_e_right_bc": {0: 0.35, 100: 0.35},  # n_e boundary condition
        # set initial condition density according to Greenwald fraction.
        "nbar": 0.85,  # line average density for initial condition
        "n_e": {0: {0.0: 1.3, 1.0: 1.0}},  # Initial electron density profile
        "normalize_n_e_to_nbar": True,  # normalize initial n_e to nbar
        "n_e_nbar_is_fGW": True,  # nbar is in units for greenwald fraction
        "initial_psi_from_j": True,  # initial psi from current formula
        "initial_j_is_total_current": True,  # only ohmic current on init
        "current_profile_nu": 2,  # exponent in initial current formula
    },
    "numerics": {
        "t_final": 150,  # length of simulation time in seconds
        "fixed_dt": 1,  # fixed timestep
        "evolve_ion_heat": True,  # solve ion heat equation
        "evolve_electron_heat": True,  # solve electron heat equation
        "evolve_current": True,  # solve current equation
        "evolve_density": True,  # solve density equation
    },
    "geometry": {
        "geometry_type": "chease",
        "geometry_file": "ITER_hybrid_citrin_equil_cheasedata.mat2cols",
        "Ip_from_parameters": True,
        "R_major": 6.2,  # major radius (R) in meters
        "a_minor": 2.0,  # minor radius (a) in meters
        "B_0": 5.3,  # Toroidal magnetic field on axis [T]
    },
    "sources": {
        # Current sources (for psi equation)
        "ecrh": {  # ECRH/ECCD (with Lin-Liu)
            "gaussian_width": 0.05,
            "gaussian_location": 0.35,
            "P_total": eccd_power,
        },
        "generic_heat": {  # Proxy for NBI heat source
            "gaussian_location": r_nbi,  # Gaussian location in normalized coordinates
            "gaussian_width": w_nbi,  # Gaussian width in normalized coordinates
            "P_total": (nbi_times, nbi_powers),  # Total heating power
            # electron heating fraction r
            "electron_heat_fraction": el_heat_fraction,
        },
        "generic_current": {  # Proxy for NBI current source
            "use_absolute_current": True,  # I_generic is total external current
            "gaussian_width": w_nbi,
            "gaussian_location": r_nbi,
            "I_generic": (nbi_times, nbi_cd),
        },
        "fusion": {},  # fusion power
        "ei_exchange": {},  # equipartition
        "ohmic": {},  # ohmic power
        "cyclotron_radiation": {},  # cyclotron radiation
        "impurity_radiation": {  # impurity radiation + bremsstrahlung
            "model_name": "mavrin_fit",
            "radiation_multiplier": 0.0,
        },
    },
    "neoclassical": {
        "bootstrap_current": {
            "bootstrap_multiplier": 1.0,
        },
    },
    "pedestal": {
        "model_name": "set_T_ped_n_ped",
        # use internal boundary condition model (for H-mode and L-mode)
        "set_pedestal": True,
        "T_i_ped": {0: 0.5, 100: 0.5, 105: 3.0},
        "T_e_ped": {0: 0.5, 100: 0.5, 105: 3.0},
        "n_e_ped_is_fGW": True,
        "n_e_ped": 0.85,  # pedestal top n_e in units of fGW
        "rho_norm_ped_top": 0.95,  # set ped top location in normalized radius
    },
    "transport": {
        "model_name": "qlknn",  # Using QLKNN_7_11 default
        # set inner core transport coefficients (ad-hoc MHD/EM transport)
        "apply_inner_patch": True,
        "D_e_inner": 0.15,
        "V_e_inner": 0.0,
        "chi_i_inner": 0.3,
        "chi_e_inner": 0.3,
        "rho_inner": 0.1,  # radius below which patch transport is applied
        # set outer core transport coefficients (L-mode near edge region)
        "apply_outer_patch": True,
        "D_e_outer": 0.1,
        "V_e_outer": 0.0,
        "chi_i_outer": 2.0,
        "chi_e_outer": 2.0,
        "rho_outer": 0.95,  # radius above which patch transport is applied
        # allowed chi and diffusivity bounds
        "chi_min": 0.05,  # minimum chi
        "chi_max": 100,  # maximum chi (can be helpful for stability)
        "D_e_min": 0.05,  # minimum electron diffusivity
        "D_e_max": 50,  # maximum electron diffusivity
        "V_e_min": -10,  # minimum electron convection
        "V_e_max": 10,  # minimum electron convection
        "smoothing_width": 0.1,
        "DV_effective": True,
        "include_ITG": True,  # to toggle ITG modes on or off
        "include_TEM": True,  # to toggle TEM modes on or off
        "include_ETG": True,  # to toggle ETG modes on or off
        "avoid_big_negative_s": False,
    },
    "solver": {
        "solver_type": "linear",  # linear solver with picard iteration
        "use_predictor_corrector": True,  # for linear solver
        "n_corrector_steps": 10,  # for linear solver
        "chi_pereverzev": 30,
        "D_pereverzev": 15,
        "use_pereverzev": True,
        #        'log_iterations': False,
    },
    "time_step_calculator": {
        "calculator_type": "fixed",
    },
}


class ReducedObservation(oh.Observation):
    def __init__(self, profiles=None, scalars=None, custom_bounds_file=None):
        variables = {
            "profiles": profiles if profiles is not None else [],
            "scalars": scalars if scalars is not None else [],
        }

        super().__init__(
            variables=variables,
            custom_bounds_filename=custom_bounds_file
        )


class IterHybridEnvPartialObservability(BaseEnv):

    def __init__(self, render_mode=None, **kwargs):

        # Set environment-specific defaults
        kwargs.setdefault("log_level", "warning")
        kwargs.setdefault("plot_config", "default")

        super().__init__(render_mode=render_mode, **kwargs)

    def _define_action_space(self):
        actions = [
            ah.IpAction(
                max=[15e6],  # 15 MA max plasma current
                ramp_rate=[0.2e6],
            ),  # 0.2 MA/s ramp rate limit
            ah.NbiAction(
                max=[33e6, 1.0, 1.0],  # 33 MW max NBI power
            ),
            ah.EcrhAction(
                max=[20e6, 1.0, 1.0],  # 20 MW max ECRH power
            ),
        ]

        return actions

    def _define_observation_space(self):
        # return AllObservation(custom_bounds_file="gymtorax/envs/iter_hybrid.json
        return ReducedObservation(
            profiles=OBS_PROFILES,
            scalars=OBS_SCALARS,
            custom_bounds_file=None
        )

    def _get_torax_config(self):
        return {
            "config": CONFIG,
            "discretization": "fixed",
            "ratio_a_sim": 1,
        }

    def _compute_reward(self, state, next_state, action):
        weight_list = [1, 1, 1, 1, 1]

        # Rewards are only provided for fusion gain when the plasma is in H-mode
        def _is_H_mode():
            if (
                next_state["profiles"]["T_e"][0] > 10
                and next_state["profiles"]["T_i"][0] > 10
            ):
                return True
            else:
                return False

        # Fusion gain Q reward: r_fusion = Q / 10
        def _r_fusion_gain():
            fusion_gain = (torax_reward.get_fusion_gain(next_state) / 10)
            if _is_H_mode():
                return fusion_gain
            else:
                return 0

        # Confinement quality H98 reward: reward H98 value up to 1 (higher H98 value = better confinement)
        def _r_h98():
            h98 = torax_reward.get_h98(next_state)
            if _is_H_mode():
                if h98 <= 1:
                    return h98
                else:
                    return 1
            else:
                return 0

        # Minimum safety factor reward: reward q_min value up to 1 (q_min < 1 leads to disruptions)
        def _r_q_min():
            q_min = torax_reward.get_q_min(next_state)
            if q_min <= 1:
                return q_min
            elif q_min > 1:
                return 1

        # Edge safety factor reward: reward q_95 value / 3 value up to 1
        def _r_q_95():
            q_95 = torax_reward.get_q95(next_state)
            if q_95 / 3 <= 1:
                return q_95 / 3
            else:
                return 1

        # Greenwald fraction reward: reward fgw value < 0.9 with 1, 0.9 - 1 with a small reward, > 1 with 0
        def _r_greenwald():
            fgw = float(next_state["scalars"]["fgw_n_e_line_avg"][0])  # Need to custom define this reward
        
            if fgw <= 0.9:
                return 1
            elif fgw <= 1:
                return 1 - (fgw - 0.9) / 0.1
            else:
                return 0

        # Calculate individual reward components
        r_fusion_gain = weight_list[0] * _r_fusion_gain() / 50
        r_h98 = weight_list[1] * _r_h98() / 50
        r_q_min = weight_list[2] * _r_q_min() / 150
        r_q_95 = weight_list[3] * _r_q_95() / 150
        r_greenwald = weight_list[4] * _r_greenwald() / 150

        total_reward = r_fusion_gain + r_h98 + r_q_min + r_q_95 + r_greenwald

        return total_reward

## Implement PPO Architecture

Our chosen algorithm is Proximal Policy Optimization, implemented using the Stable-Baselines3 (SB3) library.

### Data preprocessing

In [5]:
class FlattenToraxObservation(gym.ObservationWrapper):
    """
    Convert TORAX observation dict:
    
        {
            "profiles": {var: np.array(shape=(n_points,))},
            "scalars":  {var: np.array([value])}
        }

    into one flat 1D vector Box for use with Stable-Baselines3.
    """

    def __init__(self, env):
        super().__init__(env)

        obs_space = env.observation_space
        assert isinstance(obs_space, spaces.Dict), \
            "Expected Dict observation space from TORAX."

        # Store key order so flattening is consistent across time steps
        self.profile_keys = list(obs_space["profiles"].spaces.keys())
        self.scalar_keys = list(obs_space["scalars"].spaces.keys())

        # Compute total dimensionality
        dim = 0

        # Profiles (vectors)
        for key in self.profile_keys:
            box = obs_space["profiles"].spaces[key]
            dim += int(np.prod(box.shape))

        # Scalars
        for key in self.scalar_keys:
            box = obs_space["scalars"].spaces[key]
            dim += int(np.prod(box.shape))

        # Define new flattened observation space
        low = -np.inf * np.ones(dim, dtype=np.float32)
        high = np.inf * np.ones(dim, dtype=np.float32)
        self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)

    def observation(self, obs):
        """Flatten TORAX nested dict observation into a 1D vector."""
        parts = []

        # Profiles
        for key in self.profile_keys:
            parts.append(np.asarray(obs["profiles"][key]).ravel())

        # Scalars
        for key in self.scalar_keys:
            parts.append(np.asarray(obs["scalars"][key]).ravel())

        flat = np.concatenate(parts).astype(np.float32)
        return flat


class FlattenToraxAction(gym.ActionWrapper):
    """
    Convert TORAX action Dict {
        'Ip': Box(1,)
        'NBI': Box(3,)
        'ECRH': Box(3,)
    }
    into a single 1D Box for SB3, and unflatten on env.step().
    """

    def __init__(self, env):
        super().__init__(env)

        act_space = env.action_space
        assert isinstance(act_space, spaces.Dict), \
            "Expected Dict action space from TORAX"

        self.keys = list(act_space.spaces.keys())

        # Compute flattened dimension
        dims = []
        for k in self.keys:
            dims.append(int(np.prod(act_space[k].shape)))

        self.key_dims = dims
        total_dim = sum(dims)

        # Create the new flattened Box action space
        lows = []
        highs = []
        for k in self.keys:
            box = act_space[k]
            lows.append(box.low.flatten())
            highs.append(box.high.flatten())

        lows = np.concatenate(lows).astype(np.float32)
        highs = np.concatenate(highs).astype(np.float32)

        self.action_space = spaces.Box(low=lows, high=highs, dtype=np.float32)

    def action(self, flat_action):
        """
        Convert flat action vector back into TORAX dict for env.step().
        """
        out = {}
        idx = 0
        for k, dim in zip(self.keys, self.key_dims):
            out[k] = flat_action[idx: idx + dim]
            idx += dim
        return out


class ActionRescaleWrapper(gym.ActionWrapper):
    """
    Maps actions from [-1, 1] to the TORAX physical action ranges.
    """

    def __init__(self, env):
        super().__init__(env)

        self.low = env.action_space.low
        self.high = env.action_space.high

        # New action space presented to the agent
        self.action_space = gym.spaces.Box(
            low=-1.0, high=1.0, shape=self.low.shape, dtype=np.float32
        )

    def action(self, normalized_action):
        # Convert from [-1,1] to [low, high]
        return self.low + (normalized_action + 1.0) * 0.5 * (self.high - self.low)

In [6]:
# env = IterHybridEnvPartialObservability()
# env = FlattenToraxObservation(env)
# env = FlattenToraxAction(env)
# env = ActionRescaleWrapper(env)

# obs, info = env.reset()
# print("obs shape:", obs.shape)
# print("action space:", env.action_space)
# print("observation space:", env.observation_space)

### PPO training

In [7]:
# Environment factory
def make_env(seed: int | None = None) -> gym.Env:
    """
    Create a single wrapped ITER hybrid environment ready for SB3.
    - partial observability
    - flattened observations
    - flattened actions
    """
    env = IterHybridEnvPartialObservability(render_mode=None, store_history=False)

    # Flatten and rescale observation and action dictionaries
    env = FlattenToraxObservation(env)
    env = FlattenToraxAction(env)
    env = ActionRescaleWrapper(env)

    if seed is not None:
        env.reset(seed=seed)

    return env

In [12]:
# Training loop

# Make training and eval environment
train_env = make_env(seed=0)
eval_env = make_env(seed=1)

# Logging + checkpoints
run_name = datetime.now().strftime("ppo_iter_%Y-%m-%d_%H-%M-%S")
log_dir = os.path.join("logs", run_name)
os.makedirs(log_dir, exist_ok=True)

checkpoint_dir = os.path.join(log_dir, "checkpoints")
os.makedirs(checkpoint_dir, exist_ok=True)

tensorboard_log = os.path.join(log_dir, "tb")

# Resume training from checkpoint
# resume_checkpoint = None
resume_checkpoint = "logs/ppo_iter_2025-11-25_23-05-59/checkpoints/ppo_ckpt_100000_steps.zip"

if resume_checkpoint is not None:
    print(f"Loading model from {resume_checkpoint}")
    model = PPO.load(resume_checkpoint, env=train_env)
    continue_training = True
else:
    print("Starting NEW PPO training.")
    continue_training = False

# PPO moded trained from scratch
if not continue_training:
    model = PPO(
        policy="MlpPolicy",
        env=train_env,
        verbose=1,
        n_steps=2048,
        batch_size=64,
        gamma=0.99,
        learning_rate=3e-4,
        ent_coef=0.0,
        clip_range=0.2,
        gae_lambda=0.95,
        n_epochs=10,
        tensorboard_log=tensorboard_log,
        policy_kwargs=dict(net_arch=[256, 256]),
    )

# Resuming training
else:
    model.set_env(train_env)
    model.tensorboard_log = tensorboard_log

# Configure SB3 logger
new_logger = configure(log_dir, ["stdout", "csv", "tensorboard"])
model.set_logger(new_logger)

# Save a checkpoint every N steps
checkpoint_callback = CheckpointCallback(
    save_freq=5000,  # env steps
    save_path=checkpoint_dir,
    name_prefix="ppo_ckpt",
)

# Evaluate periodically on a separate environment
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=os.path.join(log_dir, "best_model"),
    log_path=os.path.join(log_dir, "eval"),
    eval_freq=25_000,  # How often to run eval (steps)
    n_eval_episodes=3,
    deterministic=True,
)

# Total timesteps
total_timesteps = 200_000

model.learn(
    total_timesteps=total_timesteps,
    callback=[checkpoint_callback, eval_callback],
    progress_bar=True,
    reset_num_timesteps=not continue_training,
)

# Save final model
model.save(os.path.join(log_dir, "ppo_iter_final"))

train_env.close()
eval_env.close()

Loading model from logs/ppo_iter_2025-11-25_23-05-59/checkpoints/ppo_ckpt_100000_steps.zip


FileNotFoundError: [Errno 2] No such file or directory: 'logs\\ppo_iter_2025-11-25_23-05-59\\checkpoints\\ppo_ckpt_100000_steps.zip.zip'

## Evaluate Policy Performance

Evaluate the trained RL agent. Our policy is deterministically chosen for evaluations to always choose the mean action values given a trained policy--this empirically leads to better evaluation performance. Unfortunately, the TORAX simulator is deterministic, leading to the same evaluation output when the actions are deterministically chosen as well. Training RL agents robust to noise in the simulated data would be a good line of future work.

In [8]:
def compute_physics_metrics(state):
    """Extract key physics values from the TORAX state dict."""
    return {
        "q_min": torax_reward.get_q_min(state),
        "q_95": torax_reward.get_q95(state),
        "H98": torax_reward.get_h98(state),
        "fusion_Q": torax_reward.get_fusion_gain(state),
        "fGW": float(state["scalars"]["fgw_n_e_line_avg"][0]),
    }

In [24]:
MODEL_PATH = "logs/ppo_iter_2025-11-26_09-21-05/best_model/best_model.zip"
NUM_EPISODES = 5
OUTPUT_DIR = "evaluation_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Loading PPO model from: {MODEL_PATH}")
model = PPO.load(MODEL_PATH)

# Create evaluation environment
eval_env = DummyVecEnv([lambda: make_env()])

all_episode_rewards = []
all_metrics = []

for ep in range(NUM_EPISODES):  
    seed = np.random.randint(0, 999)
    eval_env.seed(seed)
    obs = eval_env.reset()
    ep_reward = 0.0
    truncated = False

    traj_actions = []
    traj_rewards = []
    traj_metrics = []
    inference_times = []   # store per-step times

    while True:

        # Time inference
        t0 = time.perf_counter()
        action, _ = model.predict(obs, deterministic=True)
        t1 = time.perf_counter()
        inference_times.append(t1 - t0)
        
        next_obs, reward, done, info = eval_env.step(action)

        r = float(reward[0])
        d = bool(done[0])
        truncated = info[0].get("TimeLimit.truncated", False)

        # Get underlying TORAX state for metrics
        true_env = eval_env.envs[0].unwrapped
        state = true_env.state
        metrics = compute_physics_metrics(state)

        # Log
        ep_reward += r
        traj_actions.append(action[0])
        traj_rewards.append(r)
        traj_metrics.append(metrics)

        # Prepare next step
        obs = next_obs

        if d:  # terminated OR truncated
            break

    all_episode_rewards.append(ep_reward)
    all_metrics.append(traj_metrics)

    # Print episode reward
    print(f"Episode {ep+1}/{NUM_EPISODES} — Reward = {ep_reward:.3f} "
          f"{'(truncated)' if truncated else ''}")

    # Print inference time
    print(f"Mean inference time per step: {np.mean(inference_times)*1000:.3f} ms")
    print(f"Std  inference time per step: {np.std(inference_times)*1000:.3f} ms")
    print(f"Min  inference time: {np.min(inference_times)*1000:.3f} ms")
    print(f"Max  inference time: {np.max(inference_times)*1000:.3f} ms")

    # Save trajectory as CSV
    df = pd.DataFrame(traj_metrics)
    df["reward"] = traj_rewards
    csv_path = f"{OUTPUT_DIR}/episode_{ep+1}_trajectory.csv"
    df.to_csv(csv_path, index=False)
    print(f"Saved episode trajectory to: {csv_path}")

    print("")

    # Prepare data for plotting
    actions = np.array(traj_actions)
    qmin_series = [m["q_min"]    for m in traj_metrics]
    q95_series  = [m["q_95"]     for m in traj_metrics]
    fgw_series  = [m["fGW"]      for m in traj_metrics]
    Q_series    = [m["fusion_Q"] for m in traj_metrics]
    H98_series  = [m["H98"]      for m in traj_metrics]

    # Actions
    plt.figure(figsize=(10, 4), dpi=120)
    plt.title(f"Episode {ep+1}: Actions")
    plt.plot(actions[:, 0], label="Ip command")
    plt.plot(actions[:, 1], label="NBI power")
    plt.plot(actions[:, 2], label="ECRH power")
    plt.xlabel("Timestep")
    plt.legend()
    plt.savefig(f"{OUTPUT_DIR}/episode_{ep+1}_actions.png")
    plt.close()

    # q_min
    plt.figure(figsize=(10, 4), dpi=120)
    plt.title(f"Episode {ep+1}: q_min(t)")
    plt.plot(qmin_series)
    plt.axhline(1.0, color="r", linestyle="--", label="q_min = 1 limit")
    plt.xlabel("Timestep")
    plt.legend()
    plt.savefig(f"{OUTPUT_DIR}/episode_{ep+1}_qmin.png")
    plt.close()

    # q_95
    plt.figure(figsize=(10, 4), dpi=120)
    plt.title(f"Episode {ep+1}: q_95(t)")
    plt.plot(q95_series)
    plt.axhline(3.0, color="r", linestyle="--", label="q_95 = 3 limit")
    plt.xlabel("Timestep")
    plt.legend()
    plt.savefig(f"{OUTPUT_DIR}/episode_{ep+1}_q95.png")
    plt.close()

    # fGW
    plt.figure(figsize=(10, 4), dpi=120)
    plt.title(f"Episode {ep+1}: Greenwald Fraction fGW(t)")
    plt.plot(fgw_series)
    plt.axhline(0.9, color="r", linestyle="--", label="fGW = 0.9 limit")
    plt.xlabel("Timestep")
    plt.legend()
    plt.savefig(f"{OUTPUT_DIR}/episode_{ep+1}_fgw.png")
    plt.close()

    # Fusion gain Q
    plt.figure(figsize=(10, 4), dpi=120)
    plt.title(f"Episode {ep+1}: Fusion Gain Q(t)")
    plt.plot(Q_series)
    plt.xlabel("Timestep")
    plt.savefig(f"{OUTPUT_DIR}/episode_{ep+1}_fusionQ.png")
    plt.close()

    # H98
    plt.figure(figsize=(10, 4), dpi=120)
    plt.title(f"Episode {ep+1}: H98(t)")
    plt.plot(H98_series)
    plt.xlabel("Timestep")
    plt.legend()
    plt.savefig(f"{OUTPUT_DIR}/episode_{ep+1}_h98.png")
    plt.close()

# Overall summary
mean_reward = np.mean(all_episode_rewards)
std_reward  = np.std(all_episode_rewards)

print("\n======== Evaluation Summary ========")
print(f"Evaluated {NUM_EPISODES} episodes")
print(f"Mean episode reward: {mean_reward:.3f} ± {std_reward:.3f}")
print(f"Results saved to: {OUTPUT_DIR}")
print("====================================\n")

Loading PPO model from: logs/ppo_iter_2025-11-26_09-21-05/best_model/best_model.zip
Episode 1/5 — Reward = 29.339 
Mean inference time per step: 1.719 ms
Std  inference time per step: 0.442 ms
Min  inference time: 0.620 ms
Max  inference time: 3.530 ms
Saved episode trajectory to: evaluation_results/episode_1_trajectory.csv

Episode 2/5 — Reward = 29.339 
Mean inference time per step: 0.846 ms
Std  inference time per step: 0.107 ms
Min  inference time: 0.668 ms
Max  inference time: 1.476 ms
Saved episode trajectory to: evaluation_results/episode_2_trajectory.csv

Episode 3/5 — Reward = 29.339 
Mean inference time per step: 0.855 ms
Std  inference time per step: 0.082 ms
Min  inference time: 0.698 ms
Max  inference time: 1.300 ms
Saved episode trajectory to: evaluation_results/episode_3_trajectory.csv

Episode 4/5 — Reward = 29.339 
Mean inference time per step: 0.868 ms
Std  inference time per step: 0.094 ms
Min  inference time: 0.705 ms
Max  inference time: 1.411 ms
Saved episode traj